In [ ]:
import requests
from requests import Response
from pydantic import BaseModel,Field,field_validator,RootModel
import pandas as pd

youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
if youbike_url.status_code == 200:
    print("下載成功")
else:
    print("下載失敗")

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
        
class Youbike(RootModel):
    root:list[Site]

data = Youbike.model_validate_json(youbike_url.text)
all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df1=df.set_index(keys='行政區域')
df1_sum=df1.groupby(level='行政區域').sum()
df1_sum

In [ ]:
import requests
import pandas as pd
from pydantic import BaseModel, Field, RootModel


youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
res = requests.get(youbike_url)

if res.status_code == 200:
    print('下載成功')
else:
    print('下載失敗')

class Site(BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')

class YouBike(RootModel):
    root: list[Site]
    
data: YouBike = YouBike.model_validate_json(res.text)
all_sites: list[dict] = data.model_dump()

df_sites = pd.DataFrame(all_sites)
df_sites.groupby(by='行政區域')[['總數量', '可借數量', '可還數量']].sum()